In [2]:
def get_input(day):
    with open('../inputs/input%d.txt' % day) as f:
        return f.read().strip()

In [2]:
day = 1
input = get_input(day)

m = {'(': 1, ')': -1}

print(sum(m[x] for x in input))

s = 0
for i, x in enumerate(input):
    s += m[x]
    if s < 0:
        print(i+1)
        break

74
1795


In [3]:
day = 2
input = get_input(day)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2*l*w + 2*l*h + 2*w*h + min(l*w, l*h, w*h)
print(s)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2 * min(l+w, l+h, w+h) + l*w*h
print(s)

1606483
3842356


In [4]:
day = 3
input = get_input(day)

m = {
    '^': lambda x,y: (x, y+1),
    'v': lambda x,y: (x, y-1),
    '>': lambda x,y: (x+1, y),
    '<': lambda x,y: (x-1, y),
}

p = (0,0)
s = {p}
for x in input:
    p = m[x](*p)
    s.add(p)
print(len(s))

p1 = (0,0)
p2 = (0,0)
s = {p1}
for x1,x2 in zip(input[::2], input[1::2]):
    p1 = m[x1](*p1)
    p2 = m[x2](*p2)
    s.add(p1)
    s.add(p2)
print(len(s))

2081
2341


In [5]:
day = 4
input = get_input(day)

import hashlib
from itertools import count

def f(s):
    for i in count(1):
        if hashlib.md5((input + str(i)).encode('utf-8')).hexdigest().startswith(s):
            return i

print(f("00000"))

print(f("000000"))

117946
3938038


In [6]:
day = 5
input = get_input(day)

import re

def is_nice(s):
    return len(re.findall(r'[aeiou]', s)) > 2 \
           and re.search(r'(.)\1', s) \
           and not re.search(r'ab|cd|pq|xy', s)
        
print(sum(1 for x in input.split('\n') if is_nice(x)))

def is_nice2(s):
    return re.search(r'(..).*\1', s) \
           and re.search(r'(.).\1', s)
    
print(sum(1 for x in input.split('\n') if is_nice2(x)))

238
69


In [7]:
day = 6
input = get_input(day)

import re

def run_command(grid, cmd_str, fun1, fun2, fun3):
    m = re.match(r'(toggle|turn on|turn off) (\d+),(\d+) through (\d+),(\d+)', line)
    cmd = m.group(1)
    x1,y1,x2,y2 = map(int, m.group(2, 3, 4, 5))
    f = fun1 if cmd == "toggle" else fun2 if cmd == "turn on" else fun3
    for x in range(x1, x2+1):
        for y in range(y1, y2+1):
            grid[x][y] = f(grid[x][y])

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x ^ 1,
                lambda x: 1,
                lambda x: 0)

print(sum(map(sum, grid)))

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x + 2,
                lambda x: x + 1,
                lambda x: max(0, x-1))

print(sum(map(sum, grid)))

543903
14687245


In [8]:
day = 7
input = get_input(day)

import re
from collections import deque

ops = {
    'NOT': '~',
    'AND': '&',
    'OR': '|',
    'LSHIFT': '<<',
    'RSHIFT': '>>',
}

def parse(line):
    line = re.sub(r'([a-z]+)', 'nodes["\\1"]', line)
    expr, var = line.split(' -> ')
    for op, val in ops.items():
        expr = expr.replace(op, val)
    return var, expr

def eval_instr(instr, variables):
    try:
        exec(instr, {}, variables)
        return True
    except KeyError:
        return False

def get_node_value(instrs, varname):
    variables = {'nodes': {}}
    while len(instrs) > 0:
        instr = instrs.popleft()
        if not eval_instr(instr, variables):
            instrs.append(instr)
    return variables['nodes'][varname]

instrs = deque('{} = {}'.format(*parse(line)) 
               for line in  input.split('\n'))
sol1 = get_node_value(instrs, 'a')
print(sol1)

instr_map = dict(parse(line)
                 for line in  input.split('\n'))
instr_map['nodes["b"]'] = sol1
instrs = deque('{} = {}'.format(*pair) 
               for pair in instr_map.items())
sol2 = get_node_value(instrs, 'a')
print(sol2)

46065
14134


In [5]:
day = 8
input = get_input(day)

s = 0
for line in input.split('\n'):
    s += len(line)
    s -= len(eval(line))
# Part 1
print(s)

s = 0
for line in input.split('\n'):
    for c in line:
        if c in ['\\', '"']:
            s += 2
        else:
            s += 1
    s += 2 # surrounding quotes
    s -= len(line)
 
# Part 2
print(s)

1333
2046
